In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt


df = pd.read_csv("creditcard.csv")
col = "V1"

In [2]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 1. Detect Outliers

### 1.1 Detect Outliers by Visualization

In [3]:
# Visualize the outliers in three type of plots i.e. boxplots, histplot and violinplot for given dataframe and cols
def visualize_outliers(df, col, **kwargs):  
    """
    This function will help to plot visualizations for observing the outliers - 
    
    Arguments - 
    1. df = dataframe of data
    2. col = Can be list of column names, single column name string or list can be specified with argument name 'cols'
    3. plots = List of plots we want to display out of ['boxplot', 'violinplot', 'histplot']

    Multiple column names or multiple plots can be plotted at same time
    
    """
    
    # Set the theme of plots
    sns.set_theme(style="darkgrid", palette="pastel")

    # Set the list of column names and adjust with the typeof input given
    cols=[]
    if type(col)==type('str'):
        cols.append(col)
    
    if type(col)==type([]):
        cols = col
        
    if 'cols' in kwargs.keys():
        cols = kwargs['cols']
    
    # Plotting for every column name specified
    for col in cols:
        # Set the subplots and titles
        num_plots = len(kwargs['plot'])
        fig, ax = plt.subplots(nrows=num_plots, ncols=1, sharex=True, sharey=False, figsize=(11, 4*num_plots))
        plt.suptitle("Outliers in {}".format(col))
        plt.xlabel(None)
        plt.ylabel(None)
                
        # Plotting for every type of plot
        plot_num = 0
        for plot in kwargs['plot']:
            
            #If only plot to be plotted
            if num_plots==1:
                if plot=='boxplot':
                    ax.set_title("Boxplot")
                    sns.boxplot(x=df[col], linewidth=1, color='hotpink', ax=ax)
                    ax.set_xlabel(None)
                    ax.set_ylabel(None)

                if plot=='violinplot':
                    ax.set_title("Violinplot")
                    sns.violinplot(x=df[col], linewidth=1, color='tomato', ax=ax)
                    ax.set_xlabel(None)
                    ax.set_ylabel(None)

                if plot=='histplot':
                    ax.set_title("Histogram")
                    sns.histplot(x=df[col], linewidth=0.1, color='darkviolet', ax=ax, bins=50)
                    ax.set_xlabel(None)
                    ax.set_ylabel(None)
                    
            else:
                # If more than one plot
                if plot=='boxplot':
                    ax[plot_num].set_title("Boxplot")
                    sns.boxplot(x=df[col], linewidth=1, color='hotpink', ax=ax[plot_num])
                    ax[plot_num].set_xlabel(None)
                    ax[plot_num].set_ylabel(None)

                if plot=='violinplot':
                    ax[plot_num].set_title("Violinplot")
                    sns.violinplot(x=df[col], linewidth=1, color='tomato', ax=ax[plot_num])
                    ax[plot_num].set_xlabel(None)
                    ax[plot_num].set_ylabel(None)

                if plot=='histplot':
                    ax[plot_num].set_title("Histogram")
                    sns.histplot(x=df[col], linewidth=0.1, color='darkviolet', ax=ax[plot_num], bins=50)
                    ax[plot_num].set_xlabel(None)
                    ax[plot_num].set_ylabel(None)


            plot_num+=1

        plt.plot()
    
    
    
# Visulization of two variables in scatterplots for every variables    
def visualize_outliers_two_variables(df):
    """
    This function will help to plot visualizations for observing the outliers combination of variables -
    Arguments - 
    1. df = Give the dataframe

    """
    
    # Set the theme of plots
    sns.set_theme(style="darkgrid", palette="pastel")
    
    # Plotting the scatterplots for combination of two variables
    fig, ax = plt.subplots(nrows=1, ncols=1, sharex=True, sharey=True, figsize=(11, 11))
    plt.suptitle("Scatterplots for every combination of variables")
    sns.pairplot(df, ax=ax)
    plt.plot()
    


### 1.2 Detect Outliers by Z-Score

In [4]:
def outlier_by_zscore(df, col, threshold):
    """
    Function detect outliers by specifying the threashold value for Z Score
    Z Score = (datapoint - mean)/Standard deviation
    
    Threashold values are kept around 2 to 4 generally
    
    """
    # Set a threshold for outliers
    data = df[col]

    # Calculate Z-scores for the column
    z_scores = np.abs(stats.zscore(data))

    # Create a new DataFrame without outliers
    outliers_data_point = data[z_scores > threshold]
    outliers_data_point = list(outliers_data_point)

    return outliers_data_point

### 1.3 Detect Outliers by IQR Test

In [7]:
# Load your dataset into a pandas DataFrame (replace 'data.csv' with your data file)
data = df["V1"]

# Calculate Q1 and Q3
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)

# Calculate IQR
IQR = Q3 - Q1

# Define lower and upper bounds
LB = Q1 - 1.5*IQR
UB = Q3 + 1.5*IQR

# Identify and remove outliers
filtered_data = data[(data >= LB) & (data <= UB)]